In [12]:
import pandas as pd
arrest_df = pd.read_csv("data/arrests.csv")

arrest_df 

crime_df = pd.read_csv("data/crimes.csv")

crime_df

merged_arrest_crime_df = pd.merge (arrest_df, crime_df, on=['Latitude', 'Longitude'], how ='left')

merged_arrest_crime_df




,ARREST_KEY,ARREST_DATE,PD_CD_x,PD_DESC_x,KY_CD_x,OFNS_DESC_x,LAW_CODE,LAW_CAT_CD_x,ARREST_BORO,ARREST_PRECINCT,...,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD_y,Y_COORD_CD_y,Lat_Lon,New Georeferenced Column_y
0,279884335,01/03/2024,153.0,RAPE 3,104.0,RAPE,PL 1302503,F,K,77,...,BLACK,M,NaN,25-44,BLACK,F,1003509.0,185018.0,"(40.6744956865259, -73.9305713255961)",POINT (-73.9305713255961 40.6744956865259)
1,279884335,01/03/2024,153.0,RAPE 3,104.0,RAPE,PL 1302503,F,K,77,...,UNKNOWN,F,NaN,65+,WHITE,F,1003509.0,185018.0,"(40.6744956865259, -73.9305713255961)",POINT (-73.9305713255961 40.6744956865259)
2,279884335,01/03/2024,153.0,RAPE 3,104.0,RAPE,PL 1302503,F,K,77,...,BLACK,U,NaN,UNKNOWN,UNKNOWN,E,1003509.0,185018.0,"(40.6744956865259, -73.9305713255961)",POINT (-73.9305713255961 40.6744956865259)
3,279884335,01/03/2024,153.0,RAPE 3,104.0,RAPE,PL 1302503,F,K,77,...,BLACK,M,NaN,<18,BLACK,F,1003509.0,185018.0,"(40.6744956865259, -73.9305713255961)",POINT (-73.9305713255961 40.6744956865259)
4,279884335,01/03/2024,153.0,RAPE 3,104.0,RAPE,PL 1302503,F,K,77,...,BLACK,M,NaN,65+,BLACK,F,1003509.0,185018.0,"(40.6744956865259, -73.9305713255961)",POINT (-73.9305713255961 40.6744956865259)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2998251,288345759,06/11/2024,510.0,"CONTROLLED SUBSTANCE, INTENT T",117.0,DANGEROUS DRUGS,PL 2200601,F,Q,103,...,BLACK,M,NaN,UNKNOWN,UNKNOWN,E,1038891.0,192114.0,"(40.69381660456438, -73.80295796488015)",POINT (-73.80295796488015 40.69381660456438)
2998252,288345759,06/11/2024,510.0,"CONTROLLED SUBSTANCE, INTENT T",117.0,DANGEROUS DRUGS,PL 2200601,F,Q,103,...,BLACK,M,NaN,UNKNOWN,UNKNOWN,E,1038891.0,192114.0,"(40.69381660456438, -73.80295796488015)",POINT (-73.80295796488015 40.69381660456438)
2998253,288578223,06/15/2024,792.0,CRIMINAL POSSESSION WEAPON,118.0,DANGEROUS WEAPONS,PL 265031B,F,B,47,...,BLACK,M,NaN,45-64,WHITE HISPANIC,F,1024133.0,262519.0,"(40.887139, -73.855758)",POINT (-73.855758 40.887139)
2998254,288578223,06/15/2024,792.0,CRIMINAL POSSESSION WEAPON,118.0,DANGEROUS WEAPONS,PL 265031B,F,B,47,...,BLACK,M,NaN,45-64,BLACK,F,1024133.0,262519.0,"(40.887139, -73.855758)",POINT (-73.855758 40.887139)
